# param recovery for both model

In [40]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score
import optuna


# important directories

In [41]:
output_dir = "27_RL_agent_TDlearn_output_both_param_recovery"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output/model_behavior'


df_participants = []
df_colors_numbers = []


def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_excel(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".xlsx")
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [42]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    



for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    
    for df_list in [ df_colors_numbers]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type

In [43]:
for df in df_colors_numbers:
    df['model_choices'] = df['model_choices'].replace({1: 'arrowup', 0: 'arrowdown'})


In [44]:
for df in df_colors_numbers:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == 'arrowup') or (my < your and choice == 'arrowdown')):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [45]:
df_participants[2]

,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk,block_type
0,1067,uniform,894,win,3,9,3575,10.5,0,response,arrowdown,1,0,0,0.250,uniform
1,237,uniform,925,lose,2,1,903,10.0,1,response,arrowdown,1,0,1,0.125,uniform
2,231,uniform,973,win,8,2,1130,10.5,2,response,arrowup,1,0,1,0.125,uniform
3,602,uniform,768,win,7,2,537,11.0,3,response,arrowup,1,0,1,0.250,uniform
4,273,uniform,937,win,9,8,809,11.5,4,response,arrowup,1,0,1,0.000,uniform
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,567,high,787,win,9,7,1580,92.0,128,response,arrowup,4,0,0,0.000,mix
266,151,low,854,win,6,3,8541,92.5,89,response,arrowup,4,0,0,0.146,mix
267,178,low,863,win,8,5,370,93.0,77,response,arrowup,4,0,1,0.023,mix
268,328,high,770,lose,6,4,644,92.5,95,response,arrowdown,4,0,1,0.385,mix


In [46]:
df_colors_numbers[2]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,myCard,yourCard,distribution,block_type,outcome
0,arrowdown,0,10.5,10.5,"[[[-0.0004738923384741673, 0.02067374904656227...",3,9,uniform,uniform,win
1,arrowdown,0,10.0,10.0,"[[[-0.0004738923384741673, 0.02067374904656227...",2,1,uniform,uniform,lose
2,arrowup,1,10.5,10.5,"[[[-0.0004738923384741673, 0.02067374904656227...",8,2,uniform,uniform,win
3,arrowup,1,11.0,11.0,"[[[-0.0004738923384741673, 0.02067374904656227...",7,2,uniform,uniform,win
4,arrowdown,1,10.5,11.5,"[[[-0.0004738923384741673, 0.02067374904656227...",9,8,uniform,uniform,lose
...,...,...,...,...,...,...,...,...,...,...
265,arrowup,1,74.0,92.0,"[[[0.4833965827191071, 0.02067374904656227], [...",9,7,high,mix,win
266,arrowup,1,74.5,92.5,"[[[0.4833965827191071, 0.02067374904656227], [...",6,3,low,mix,win
267,arrowdown,1,74.0,93.0,"[[[0.4833965827191071, 0.02067374904656227], [...",8,5,low,mix,lose
268,arrowdown,0,73.5,92.5,"[[[0.4833965827191071, 0.02067374904656227], [...",6,4,high,mix,lose


In [47]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path_participants) if file.endswith('.xlsx')]

In [48]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

# policy_table = percentage_matrix 


Q_table_init = np.random.normal(0, 0.01, (len(card_numbers), len(distributions_map), len(actions)))


Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

# print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))




 Q_table: 
 (9, 3, 2)


In [49]:
def epsilon_greedy(Q_values, epsilon):    
    probs = np.full_like(Q_values, epsilon / Q_values.shape[-1], dtype=float)  # initialize with exploration probability
    best_actions = np.argmax(Q_values, axis=-1)  # find the best action for each state
    np.put_along_axis(probs, np.expand_dims(best_actions, axis=-1), 1 - epsilon + (epsilon / Q_values.shape[-1]), axis=-1)
    return probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["model_choices"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = epsilon_greedy(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [50]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 1
alpha_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [ ]:
BIC_models = []
AIC_models = []
best_alpha_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(df_colors_numbers):
    print(f"Processing participant {idx + 1} of {len(df_colors_numbers)}")
    Q_init_participant = Q_table.copy()
    
    def objective(trial):
        alpha = trial.suggest_float("alpha", alpha_min, alpha_max)
        beta  = trial.suggest_float("beta", beta_min, beta_max)

        # negative log-likelihood (Optuna minimises)
        _, _, ll = compute_log_likelihood(alpha, beta,
                                        df_all,
                                        Q_init_participant.copy())
        return -ll

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=400, n_jobs=-1)


    best_alpha = study.best_params["alpha"]
    best_beta  = study.best_params["beta"]
    best_log_likelihood = -study.best_value

    # keep this for plotting later
    results_df = study.trials_dataframe()
    results_df["alpha"] = results_df["params_alpha"]
    results_df["beta"]  = results_df["params_beta"]
    results_df["log_likelihood"] = -results_df["value"]

    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = epsilon_greedy(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choice = int(p_arrowup >= 0.5)  # 1 if up-prob ≥ 0.5 else 0
        predicted_choices.append(predicted_choice)

    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
   
       # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

    

    # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 


    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    print(best_alpha)
    print(best_beta)
    
    # saving models evaluation variables:
    best_alpha_models.append(best_alpha)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)





[I 2025-06-12 14:55:12,733] A new study created in memory with name: no-name-1244a68a-88dc-4927-8f23-620ef0f7946e


Processing participant 1 of 35


[I 2025-06-12 14:55:14,111] Trial 16 finished with value: 193.91441280353115 and parameters: {'alpha': 0.4634476515106311, 'beta': 0.39561946486890187}. Best is trial 16 with value: 193.91441280353115.
[I 2025-06-12 14:55:14,115] Trial 23 finished with value: 287.14306085992007 and parameters: {'alpha': 0.29409596929424364, 'beta': 0.0902689478008715}. Best is trial 16 with value: 193.91441280353115.
[I 2025-06-12 14:55:14,133] Trial 9 finished with value: 186.5567193517501 and parameters: {'alpha': 0.04234987474145468, 'beta': 0.9938975566919477}. Best is trial 9 with value: 186.5567193517501.
[I 2025-06-12 14:55:14,160] Trial 10 finished with value: 176.39563195965098 and parameters: {'alpha': 0.9282301169192192, 'beta': 0.8116804096801712}. Best is trial 10 with value: 176.39563195965098.
[I 2025-06-12 14:55:14,166] Trial 11 finished with value: 175.59515173375365 and parameters: {'alpha': 0.7768010863544379, 'beta': 0.7836215246479441}. Best is trial 11 with value: 175.595151733753

0.03373654777606379
0.6218765223229875
Processing participant 2 of 35


[I 2025-06-12 14:55:36,202] Trial 23 finished with value: 156.5096737987801 and parameters: {'alpha': 0.7948852208923174, 'beta': 0.6294136711648202}. Best is trial 23 with value: 156.5096737987801.
[I 2025-06-12 14:55:36,744] Trial 6 finished with value: 283.2934180484311 and parameters: {'alpha': 0.6085109680639348, 'beta': 0.036855135629136525}. Best is trial 23 with value: 156.5096737987801.
[I 2025-06-12 14:55:36,755] Trial 11 finished with value: 161.42759193108432 and parameters: {'alpha': 0.18541145599609782, 'beta': 0.23076023398619092}. Best is trial 23 with value: 156.5096737987801.
[I 2025-06-12 14:55:36,760] Trial 15 finished with value: 181.8106660881835 and parameters: {'alpha': 0.7394816211699851, 'beta': 0.9570381945049871}. Best is trial 23 with value: 156.5096737987801.
[I 2025-06-12 14:55:36,762] Trial 19 finished with value: 151.34643026783638 and parameters: {'alpha': 0.42825732695080254, 'beta': 0.4771183767585582}. Best is trial 19 with value: 151.34643026783638

0.1534236247697103
0.46620153358239674
Processing participant 3 of 35


[I 2025-06-12 14:55:59,795] Trial 0 finished with value: 165.34693422804446 and parameters: {'alpha': 0.4672315290554637, 'beta': 0.4753089940697363}. Best is trial 0 with value: 165.34693422804446.
[I 2025-06-12 14:55:59,799] Trial 5 finished with value: 232.02608399201583 and parameters: {'alpha': 0.11843399452236344, 'beta': 0.11873048735345813}. Best is trial 0 with value: 165.34693422804446.
[I 2025-06-12 14:55:59,811] Trial 15 finished with value: 281.14135016978025 and parameters: {'alpha': 0.31578309094207013, 'beta': 0.06735345814019725}. Best is trial 0 with value: 165.34693422804446.
[I 2025-06-12 14:55:59,815] Trial 27 finished with value: 170.35164711918395 and parameters: {'alpha': 0.4858202612473093, 'beta': 0.46889975331656697}. Best is trial 0 with value: 165.34693422804446.
[I 2025-06-12 14:55:59,824] Trial 2 finished with value: 168.92930656636395 and parameters: {'alpha': 0.6908140093487877, 'beta': 0.5266690577643619}. Best is trial 0 with value: 165.34693422804446

0.2324827026979792
0.5769910956286585
Processing participant 4 of 35


# now saving the model evaluation values

In [ ]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha": best_alpha_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir, "models_evaluation_greedy.csv")
df_models_evaluation.to_csv(file_path, index=False)